In [135]:
import yfinance as yf
import seaborn as sns
import pandas  as pd
import numpy as np
import pandas_market_calendars as mcal
import pytz

import datetime
import plotly.graph_objects as go
import plotly.express as px
from FMNM.BS_pricer import BS_pricer
from FMNM.Parameters import Option_param
from FMNM.Processes import Diffusion_process

import scipy.stats as ss
from scipy.integrate import quad
from typing import List, Optional
from scipy.stats import lognorm
from scipy.stats import norm
from functools import partial

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

%matplotlib inline

from utils.gbm import GBMAnalyzer
from utils.dataFetcher import OptionsDataFetcher as odf

In [136]:
fetcher = odf('TSLA')
stock_data = fetcher.fetch_stock_data("2022-04-18", "2025-05-18")
opts_data = fetcher.fetch_options_chain()
opts_data_p = fetcher.fetch_options_chain(puts_only = True)
opts_data_c = fetcher.fetch_options_chain(calls_only = True)
# fetcher.save(stock_data, opts_data, path="retrivedData", save_stock=True, save_options=True)

In [137]:
nyse = mcal.get_calendar('NYSE')
ny_tz = pytz.timezone('America/New_York')

def trading_day_after_n_days(start_date, n_days):
    """
    Returns the first NYSE trading day on or after (or on or before, if n_days < 0)
    the date shifted by n_days. Always returns a trading day >= or <= the shifted date.
    """
    # Shift the date by n_days
    target_date = (start_date + pd.Timedelta(days=n_days)).normalize()

    # Define search window (inclusive)
    if n_days >= 0:
        search_start = target_date
        search_end = target_date + pd.Timedelta(days=15)
    else:
        search_start = target_date - pd.Timedelta(days=15)
        search_end = target_date

    # Get valid trading days in the range
    schedule = nyse.valid_days(start_date=search_start, end_date=search_end)

    if not schedule.empty:
        selected_date = schedule[0] if n_days >= 0 else schedule[-1]
        return selected_date.tz_convert(ny_tz)
    else:
        raise ValueError(f"No trading day found in the expected window around {target_date}")

In [138]:
def mu_sig_annual(prices, t, days = 365):

    # Step 1: Calculate daily log returns
    log_returns = np.log(prices[1:] / prices[:-1])
    
    # Step 2: Calculate daily mean return and daily volatility
    mu_daily = np.mean(log_returns)
    sigma_daily = np.std(log_returns)
    
    # Step 3: Annualize drift and volatility assuming 252 trading days
    mu = mu_daily * days * t
    sigma = sigma_daily * np.sqrt(days * t)
    return mu, sigma

def drift_term(mu, sigma, t):
    return (mu - 0.5 * sigma**2) * t

def St(S0, mu, sigma, t):
    Z = np.random.normal()  # standard normal random variable
    return S0 * np.exp(drift_term(mu, sigma, t) + sigma * Z * np.sqrt(t))

def predict_prices(prices, t, days = 365):
    mu, sigma = mu_sig_annual(prices, t, days)
    predicted = []
    for S0 in prices:
        predicted_price = St(S0, mu, sigma, t)
        predicted.append(predicted_price)
    return predicted

In [222]:
def mu_sig_annual(prices, days=252):
    log_returns = np.log(prices[1:] / prices[:-1])
    mu_daily = np.mean(log_returns)
    sigma_daily = np.std(log_returns)
    mu = mu_daily * days
    sigma = sigma_daily * np.sqrt(days)
    return mu, sigma

def simulate_gbm_path(S0, mu, sigma, t, steps=252):
    dt = t / steps
    prices = [S0]
    for _ in range(steps):
        Z = np.random.normal()
        S_next = prices[-1] * np.exp((mu - 0.5 * sigma**2) * dt + sigma * Z * np.sqrt(dt))
        prices.append(S_next)
    return prices

def predict_prices(prices, t, steps=252):
    mu, sigma = mu_sig_annual(prices)
    S0 = prices[-1]
    return simulate_gbm_path(S0, mu, sigma, t, steps)


In [223]:
end_date_train = stock_data.index[300]
t = 1
days = 252
start_date_train = trading_day_after_n_days(end_date_train, -t*days)
start_date_test = trading_day_after_n_days(end_date_train,0)
end_date_test = trading_day_after_n_days(start_date_test,t*days)

In [224]:
train_data = stock_data.loc[start_date_train:end_date_train]
test_data = stock_data.loc[start_date_test:end_date_test]

In [225]:
initial_dates = np.array(train_data.index)
pred_dates = [trading_day_after_n_days(date, t*days) for date in initial_dates]